#### This file aggregates the data provided by Kaggle and the web scraped Pomeroy data

In [ ]:
import pandas as pd
import numpy as np
kaggleData = "input/"

df_results = pd.read_csv(kaggleData + "RegularSeasonCompactResults.csv")
print(df_tourney.shape)
df_results.head()

drop season results before 2002 since we dont have any data on those from the pomeroy data set

In [ ]:
df_results.drop(df_season.index[:69493], inplace=True)
df_results.index = np.arange(0, df_season.shape[0])

In [ ]:
df_results

In [ ]:
df_results["WAdjEM"] = -1
df_results["WAdjO"] = -1
df_results["WAdjD"] = -1
df_results["WAdjT"] = -1

In [ ]:
for year in range(2002, 2019):
    if year != 2018:
        exec('df_{} = pd.read_csv(kaggleData + "{}_final.csv")'.format(year, year))
    else:
        exec('df_{} = pd.read_csv(kaggleData + "{}.csv")'.format(year, year))

In [ ]:
for i in range(df_results.shape[0]):
    season, WTeamID = df_results["Season"].loc[i], df_results["WTeamID"].loc[i]
    exec('row = df_{}.loc[df_{}["TeamID"] == WTeamID]'.format(season, season))
    if row.empty:
        continue
    else:
        df_results["WAdjEM"].loc[i] =  row.AdjEM.values[0]
        df_results["WAdjO"].loc[i] =  row.AdjO.values[0]
        df_results["WAdjD"].loc[i] =  row.AdjD.values[0]
        df_results["WAdjT"].loc[i] =  row.AdjT.values[0]
    if i % 10000 == 0:
        print(str(i) + "iterations")
    

In [ ]:
df_results["LAdjEM"] = -1
df_results["LAdjO"] = -1
df_results["LAdjD"] = -1
df_results["LAdjT"] = -1

In [ ]:
for i in range(df_results.shape[0]):
    season, LTeamID = df_results["Season"].loc[i], df_results["LTeamID"].loc[i]
    exec('row = df_{}.loc[df_{}["TeamID"] == LTeamID]'.format(season, season))
    if row.empty:
        continue
    else:
        df_results["LAdjEM"].loc[i] =  row.AdjEM.values[0]
        df_results["LAdjO"].loc[i] =  row.AdjO.values[0]
        df_results["LAdjD"].loc[i] =  row.AdjD.values[0]
        df_results["LAdjT"].loc[i] =  row.AdjT.values[0]
    if i % 5000 == 0:
        print(str(i) + " iterations")
    

In [ ]:
df_results

In [ ]:
df_results = pd.read_csv(kaggleData + "training.csv")
df_results.drop(["Unnamed: 0", "DayNum", "NumOT", "WScore", "LScore"], axis=1, inplace=True)

In [ ]:
df_winners, df_losers = df_results.copy(), df_results.copy()

In [ ]:
df_winners["AdjEM"] = df_winners["WAdjEM"] - df_winners["LAdjEM"]
df_winners["AdjO"] = df_winners["WAdjO"] - df_winners["LAdjO"]
df_winners["AdjD"] = df_winners["WAdjD"] - df_winners["LAdjD"]
df_winners["AdjT"] = df_winners["WAdjT"] - df_winners["LAdjT"]
df_winners.drop(["WAdjEM", "WAdjO", "WAdjD", "WAdjT", "LAdjEM", "LAdjO", "LAdjD", "LAdjT", "LTeamID"], axis=1, inplace=True)

In [ ]:
df_winners["Result"] = 1
df_winners

In [ ]:
df_losers["AdjEM"] = -(df_losers["WAdjEM"] - df_losers["LAdjEM"])
df_losers["AdjO"] = -(df_losers["WAdjO"] - df_losers["LAdjO"])
df_losers["AdjD"] = -(df_losers["WAdjD"] - df_losers["LAdjD"])
df_losers["AdjT"] = -(df_losers["WAdjT"] - df_losers["LAdjT"])
df_losers.drop(["WAdjEM", "WAdjO", "WAdjD", "WAdjT", "LAdjEM", "LAdjO", "LAdjD", "LAdjT", "WTeamID"], axis=1, inplace=True)
df_losers["Result"] = 0

In [ ]:
for i in range(df_losers.shape[0]):
    if df_losers["WLoc"].loc[i] == "N":
        continue
    
    if df_losers["WLoc"].loc[i] == "H":
        df_losers["WLoc"].loc[i] = "A"
    else:
        df_losers["WLoc"].loc[i] = "H"

In [ ]:
df_predictions = pd.concat((df_winners, df_losers))

In [ ]:
df_predictions.drop(["LTeamID", "WTeamID"], axis=1, inplace=True)

In [ ]:
df_predictions.to_csv(kaggleData + "Predictions.csv")

In [ ]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.utils import shuffle

clf = LogisticRegressionCV(cv=5, Cs=8, n_jobs=-1, scoring="neg_log_loss")

In [ ]:
X_train = df_predictions[["AdjEM", "AdjO", "AdjD", "AdjT"]]
y_train = df_predictions["Result"]

In [ ]:
X_train, y_train = shuffle(X_train, y_train)
clf.fit(X_train, y_train)